In [2]:
import sevenbridges as sbg
from sevenbridges.errors import SbgError
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
import sys
import re
import pdb
import concurrent.futures
from requests import request
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper], advance_access=True)

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

### get cost

In [5]:
def parse_task(prefix, id_prefix, suffix, task):
    # pdb.set_trace()
    try:
        if re.search(prefix, task.name):
            # print (task.id + '\t' + task.name + '\t' + str(task.price.amount) + '\n')
            return task.id + '\t' + task.name + '\t' + str(task.price.amount) + '\t' + task.app + '\n'
    except Exception as e:
        sys.stderr.write('Could not process task ' + task.id + ' ' + task.name + ' got error ' + str(e) + '\n')
        sys.exit()

In [6]:
pre = 'STAR FUSION'
id_pre = 'BS_'
suffix = ''
project = 'd3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study'

tasks = api.tasks.query(project=project, status='COMPLETED').all()
#task = next(tasks)
#parse_task(pre, id_pre, suffix, task)
#pdb.set_trace()
out_fh = open('/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/cost_est/star-fusion_cost.txt', 'w')

with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(parse_task, pre, id_pre, suffix, task): task for task in tasks}
    for cost_info in concurrent.futures.as_completed(results):
        try:
            if cost_info.result() is not None:
                out_fh.write(cost_info.result())
        except Exception as e:
            print (e)
out_fh.close()


### get exact run time

In [3]:
def merge_and_sum_time_intervals(prefix, id_prefix, suffix, task):
    data = []
    if re.search(prefix, task.name):
        for job in task.get_execution_details().jobs:
            try:
                pair = (job.start_time, job.end_time)
                data.append(pair)
            except Exception as e:
                print (e)
                pdb.set_trace()
                hold = 1
        data = sorted(data, key=lambda x: x[0])


        # from https://stackoverflow.com/questions/34797525/how-to-correctly-merge-overlapping-datetime-ranges-in-python
        result = []
        try:
            t_old = data[0]
            for t in data[1:]:
                if t_old[1] >= t[0]:  #I assume that the data is sorted already
                    t_old = ((min(t_old[0], t[0]), max(t_old[1], t[1])))
                else:
                    result.append(t_old)
                    t_old = t

            else:
                result.append(t_old)
        except Exception as e:
            print(e)
            pdb.set_trace()
            hold = 1
        total_seconds = 0
        for t_int in result:
            total_seconds += (t_int[1] - t_int[0]).seconds
        #print ('Task ran in ' + str(total_seconds) + ', which is ' + str(total_seconds/3600) + ' hours')
        return task.id + '\t' + task.name + '\t' + str(total_seconds/3600) + '\n'
    

In [4]:
# use task.get_execution_details().jobs - is a list that will giev you start and end time of each job, can use to create opverlapping tim e intervals, then bin and sum
pre = 'STAR FUSION'
id_pre = 'BS_'
suffix = 'INPUT'
project = 'd3b-bixu/rs-vpf5jbc3-cov-irt-controlled-access-study'

tasks = api.tasks.query(project=project, status='COMPLETED').all()
# task = next(tasks)
# merge_and_sum_time_intervals(task)
out_fh = open('/Users/brownm28/Documents/2020-Apr-8_RNAseq_snv_dev/cost_est/star-fusion_run.txt', 'w')

with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(merge_and_sum_time_intervals, pre, id_pre, suffix, task): task for task in tasks}
    for time_info in concurrent.futures.as_completed(results):
        try:
            if time_info.result() is not None:
                out_fh.write(time_info.result())
        except Exception as e:
            print (e)
out_fh.close()


In [ ]:
project = 'zhangb1/kf-somatic-tools-test'
task = api.tasks.get('7c7bb42f-089f-4d9e-b182-dd731de56003')
pdb.set_trace()
hold=1


In [ ]:
## Get task log

In [ ]:
tid = '575f8184-f6b1-4191-b119-1672b75d8d61'
project = 'zhangb1/kf-somatic-tools-test'
task = api.tasks.get(tid)
for job in task.get_execution_details().jobs:
    if re.search('seg', job.name):
        print (job.logs['cmd.log'].content())

### Get job run times

In [ ]:
task = api.tasks.get("7174d826-422d-4577-a288-3c6adc1f6f5a")
phrase = "vardict"
for job in task.get_execution_details().jobs:
    if job.status == "COMPLETED" and re.search(phrase, job.name):
        # pdb.set_trace()
        try:
            print (job.name + "\t" + str((job.end_time - job.start_time).seconds/3600))
        except Exception as e:
            print (e)
            pdb.set_trace()
#hold=1

### Get instance run times

In [ ]:
task = api.tasks.get("8acc7912-5380-4db0-8edc-a21ed93833d0")
phrase = "vardict_\d+"
data = {}
for job in task.get_execution_details().jobs:
    if job.status == "COMPLETED" and re.search(phrase, job.name):
        try:
            # print (job.name + "\t" + str((job.end_time - job.start_time).seconds/3600))
            sys.stderr.write(job.name + '\n')
            vm = job.instance.id
            if vm not in data:
                data[vm] = []
            pair = (job.start_time, job.end_time)
            data[vm].append(pair)
        except Exception as e:
            print (e)
            pdb.set_trace()
            hold = 1
total_hrs = 0
c5_9_spot = 0.5164
c5_9_direct = 1.53
m4_10_spot = 0.412
m4_10_direct = 2
spot_rate = c5_9_spot
direct_rate = c5_9_direct
for vm in data:
    data[vm] = sorted(data[vm], key=lambda x: x[0])
    result = []
    try:
        t_old = data[vm][0]
        for t in data[vm][1:]:
            if t_old[1] >= t[0]:  #I assume that the data is sorted already
                t_old = ((min(t_old[0], t[0]), max(t_old[1], t[1])))
            else:
                result.append(t_old)
                t_old = t

        else:
            result.append(t_old)
    except Exception as e:
        print(e)
        pdb.set_trace()
        hold = 1
    total_seconds = 0
    for t_int in result:
        total_seconds += (t_int[1] - t_int[0]).seconds
    #print ('Task ran in ' + str(total_seconds) + ', which is ' + str(total_seconds/3600) + ' hours')
    print(vm + '\t' + '\t' + str(total_seconds/3600))
    total_hrs += total_seconds/3600
print ("All spot\t" + str(total_hrs * spot_rate) + "\t" + str(total_hrs))
print ("All direct\t" + str(total_hrs * direct_rate))
    

## Get task input file IDs

In [ ]:
phrase = "kfdrc-combined-CNV"
project= "kfdrc-harmonization/sd-bhjxbdqk-08"
in_keys = ["b_allele", "input_normal_aligned", "input_tumor_aligned"]
out = open("/Users/brownm28/Documents/playground/cbttc_task_check.txt", "w")
out.write("Task ID\tTask name\tTask status\tb allele bs id\tb allele pt id\tnormal bs id\tnormal pt id\ttumor bs id\ttumor pt id\tpaired vcf N bs id\tpaired vcf T bs id\tpaired vcf pt id\n")
for task in api.tasks.query(project=project).all():
    if re.search(phrase, task.name) and (task.status == "COMPLETED" or task.status == "FAILED"):
        info = []
        try:
            for key in in_keys:
                info.append(task.inputs[key].metadata['Kids First Biospecimen ID'])
                info.append(task.inputs[key].metadata['Kids First Participant ID'])
            info.append(task.inputs['paired_vcf'].metadata['Kids First Biospecimen ID Normal'])
            info.append(task.inputs['paired_vcf'].metadata['Kids First Biospecimen ID Tumor'])
            info.append(task.inputs['paired_vcf'].metadata['Kids First Participant ID'])
            out.write(task.id + "\t" + task.name + "\t" + task.status + "\t" + "\t".join(info) + "\n")
        except Exception as e:
            print (e)
            pdb.set_trace()
            print(task.name + "\t" + task.id + "\tstatus: " + task.status + " failed parsing. skipping!")
out.close()

# Get Task Job err log info

In [ ]:
def get_task_info(task):
    if re.search(phrase, task.name):
        tname = task.name
        tstatus = task.status
        key = task.id + "\t" + tname + "\t" + tstatus
        task_list.append(key)
        return 0
    else:
        return 1

In [ ]:
project = 'kfdrc-harmonization/sd-p445achv-02'
phrase = "Vardict-Somatic"
pwd = "/Users/brownm28/Documents/2019-Nov-4_job_err_log_audit/"
out_fn = 'boyd_vardict_task_info.txt'
print (str(api.remaining) + " remaining tasks")
out = open(pwd + out_fn, 'w')
tasks = api.tasks.query(project=project).all()
task_list = []
valid = 1
n = 50
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    results = {executor.submit(get_task_info, task): task for task in tasks}
    for task_info in concurrent.futures.as_completed(results):
        if (valid % n) == 0:
            sys.stderr.write("Processed " + str(valid) + " valid tasks\n")
            print (api.remaining)
        if task_info.result() == 0:
            valid += 1
# for task in tasks:
#     parse_job_info(task)

out.write("Task ID\tTask Name\tTask Status\n")
for info in task_list:
    out.write(info + "\n")
out.close()


In [ ]:
def parse_bulk_info(task):
    cur_task = task.resource
    tid = cur_task.id
    job_dict[tid] = []
    jphrase = 'vardict_\d+_s'
    for job in cur_task.get_execution_details().jobs:
        if re.search(jphrase, job.name):
            # pdb.set_trace()
            job_dict[tid].append([job.name, job.status, job.logs['job.err.log'].id, str(job.logs['job.err.log'].size)])


In [ ]:
pwd = "/Users/brownm28/Documents/2019-Nov-4_job_err_log_audit/"
manifest_fn = 'boyd_vardict_task_info.txt'
task_manifest = open(pwd + manifest_fn)
head = next(task_manifest)
task_ids = []
for line in task_manifest:
    info = line.rstrip('\n').split('\t')
    task_ids.append(info[0])
task_manifest.close()
task_bulk = []
x = 1
max_j = 50
total = len(task_ids)
for i in range(0, total, max_j):
    uset = i + max_j
    if uset > total:
        uset = total
    task_bulk.extend(api.tasks.bulk_get(tasks=task_ids[i:uset]))
project = 'kfdrc-harmonization/sd-p445achv-02'
phrase = "Vardict-Somatic"

out_fn = 'boyd_vardict_job_info.txt'
out = open(pwd + out_fn, 'w')
out.write("Task ID\tJob Name\tJob Status\tError log ID\tError log size\n")
set_n = 1
for i in range(0, total, max_j):
    uset = i + max_j
    if uset > total:
        uset = total
    job_dict = {}
    sys.stderr.write("Processing set " + str(set_n) + ", api calls remaining: " + str(api.remaining) + "\n")
#     for task in task_bulk[i:uset]:
#         parse_bulk_info(task)
    with concurrent.futures.ThreadPoolExecutor(8) as executor:
        results = {executor.submit(parse_bulk_info, task): task for task in task_bulk[i:uset]}
    try:
        for tid in job_dict:
            for job in job_dict[tid]:
                out.write(tid + "\t" + "\t".join(job) + "\n")
        out.flush()
        sys.stderr.write("Finished with set " + str(set_n) + ", api calls remaining: " + str(api.remaining) + '\n')
        set_n += 1
    except Exception as e:
        sys.stderr.write(str(e) + "\n")
        pdb.set_trace()
        hold = 1

out.close()

    

In [ ]:
task_list = ['72e5103a-4491-4073-a615-cd830ec63690', 'fe5aeb6d-a72d-4319-b8de-12083684376e']
task_bulk = api.tasks.bulk_get(tasks=task_list)
pdb.set_trace()
pause = 1

## Get wf used in task

In [ ]:
pwd = "/Users/brownm28/Documents/2019-Nov-4_job_err_log_audit/to_check/"
manifest_fn = 'pbta.txt'
task_manifest = open(pwd + manifest_fn)
out_fn = 'pbta_vardict_app.txt'
out = open(pwd + out_fn, 'w')
head = next(task_manifest)
for line in task_manifest:
    info = line.rstrip('\n').split('\t')
    task = api.tasks.get(info[0])
    out.write(task.id + "\t" + task.app + "\n")
out.close()

## Fix swapped inputs

In [2]:
def check_fix_swap(task):
    try:
        if re.search('LANCET_TCGA', task.name):
            if not re.search('-10A-', task.inputs['input_normal_aligned'].name):
                sys.stderr.write('Inputs swapped, re-tasking ' + task.name + '\n')
                task_name = 'LANCET: ' + task.inputs['input_tumor_name'] + " " + task.inputs['input_normal_name']
                in_dict = {}
                for key in task.inputs:
                    in_dict[key] = task.inputs[key]
                in_dict['input_tumor_aligned'] = task.inputs['input_normal_aligned']
                in_dict['input_normal_aligned'] = task.inputs['input_tumor_aligned']
                task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=in_dict, run=False)
                task.inputs['output_basename'] = task.id
                task.save()
    except Exception as e:
        sys.stderr.write(str(e) + "\nProcessing " + task.name + " failed\n")
        #pdb.set_trace()
        hold = 1
        sys.exit()
        

In [3]:
project = 'cavatica/openpbta-tcga'
app_name = project + "/kfdrc-lancet-wf-baminput"
tasks = api.tasks.query(project=project, status="COMPLETED").all()
i = 1
n = 50

with concurrent.futures.ThreadPoolExecutor(8) as executor:
    results = {executor.submit(check_fix_swap, task): task for task in tasks}
    for result in concurrent.futures.as_completed(results):
        if i % n == 0:
            sys.stderr.write(str(i) + ' tasks set up\n')
        i += 1
# for task in tasks:
#     check_fix_swap(task)

Inputs swapped, re-tasking LANCET_TCGA-S9-A7R3-01A-11D-A34J-08_TCGA-S9-A7R3-10A-01D-A34M-08
Inputs swapped, re-tasking LANCET_TCGA-HW-7490-01A-11D-2024-08_TCGA-HW-7490-10A-01D-2024-08
Inputs swapped, re-tasking LANCET_TCGA-TM-A7C4-01A-11D-A32B-08_TCGA-TM-A7C4-10A-01D-A329-08
Inputs swapped, re-tasking LANCET_TCGA-CS-4944-01A-01D-1468-08_TCGA-CS-4944-10A-01D-1468-08
Inputs swapped, re-tasking LANCET_TCGA-P5-A5EW-01A-11D-A27K-08_TCGA-P5-A5EW-10A-01D-A27N-08
Inputs swapped, re-tasking LANCET_TCGA-HT-7680-01A-11D-2253-08_TCGA-HT-7680-10A-01D-2253-08
Inputs swapped, re-tasking LANCET_TCGA-S9-A6U5-01A-12D-A33T-08_TCGA-S9-A6U5-10A-01D-A33W-08
Inputs swapped, re-tasking LANCET_TCGA-CS-6667-01A-12D-2024-08_TCGA-CS-6667-10A-01D-2024-08
50 tasks set up
100 tasks set up
150 tasks set up
Inputs swapped, re-tasking LANCET_TCGA-S9-A7R7-01A-11D-A34J-08_TCGA-S9-A7R7-10A-01D-A34M-08
Inputs swapped, re-tasking LANCET_TCGA-CS-4938-01B-11D-1893-08_TCGA-CS-4938-10A-01D-1893-08
Inputs swapped, re-tasking LAN

Inputs swapped, re-tasking LANCET_TCGA-DU-5874-01A-11D-1705-08_TCGA-DU-5874-10A-01D-1705-08
Inputs swapped, re-tasking LANCET_TCGA-DU-A7TA-01A-11D-A33T-08_TCGA-DU-A7TA-10A-01D-A33W-08
Inputs swapped, re-tasking LANCET_TCGA-DU-A5TS-01A-11D-A289-08_TCGA-DU-A5TS-10A-01D-A289-08
Inputs swapped, re-tasking LANCET_TCGA-HT-7481-01A-11D-2024-08_TCGA-HT-7481-10A-01D-2024-08
Inputs swapped, re-tasking LANCET_TCGA-HT-7881-01A-11D-2395-08_TCGA-HT-7881-10A-01D-2396-08
Inputs swapped, re-tasking LANCET_TCGA-DU-A6S2-01A-21D-A32B-08_TCGA-DU-A6S2-10A-01D-A329-08
Inputs swapped, re-tasking LANCET_TCGA-FG-5964-01A-11D-1705-08_TCGA-FG-5964-10A-01D-1705-08
Inputs swapped, re-tasking LANCET_TCGA-DU-8164-01A-11D-2253-08_TCGA-DU-8164-10A-01D-2253-08
250 tasks set up
Inputs swapped, re-tasking LANCET_TCGA-DU-7294-01A-11D-2024-08_TCGA-DU-7294-10A-01D-2024-08
Inputs swapped, re-tasking LANCET_TCGA-S9-A7J3-01A-21D-A34J-08_TCGA-S9-A7J3-10A-01D-A34M-08
Inputs swapped, re-tasking LANCET_TCGA-HT-7677-01A-11D-2253-08_

### Re-draft failed task in another project

In [5]:
old_task = '416aea5b-0722-45db-9b40-8a58ed685769'
old_task_info = api.tasks.get(old_task)
project = 'zhangb1/kf-somatic-tools-test'
app_name = project + "/kfdrc-combined-somatic-wgs-cnv-wf"

in_dict = {}
for key in old_task_info.inputs:
    in_dict[key] = old_task_info.inputs[key]
    try:
        in_dict[key].copy(project=project)
    except Exception as e:
        sys.stderr.write(str(e) + "\n")
in_dict['output_basename'] = 'theta2_fail_catch_test'
task_name = "Theta2 Fail Catch Test"
task = api.tasks.create(name=task_name, project=project, app=app_name, inputs=in_dict, run=False)
task.save()

'str' object has no attribute 'copy'
'str' object has no attribute 'copy'
'NoneType' object has no attribute 'copy'
'str' object has no attribute 'copy'
'float' object has no attribute 'copy'
'str' object has no attribute 'copy'
'str' object has no attribute 'copy'
'float' object has no attribute 'copy'
'int' object has no attribute 'copy'
'str' object has no attribute 'copy'
'str' object has no attribute 'copy'
'bool' object has no attribute 'copy'
'NoneType' object has no attribute 'copy'
copy() takes no keyword arguments
'str' object has no attribute 'copy'
'str' object has no attribute 'copy'
'NoneType' object has no attribute 'copy'


<Task: id=412e7fc9-94b2-43ab-bd58-4079337407e4>

In [8]:
task_id = 'fa9f548e-5dfb-4562-a940-8c81cb5a428b'
task_info = api.tasks.get(task_id)
pdb.set_trace()
hold=1

--Return--
> <ipython-input-8-290aa3214fc1>(3)<module>()->None
-> pdb.set_trace()
(Pdb) p dir(task_info)
['_API', '_URL', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_data', '_dirty', '_fields', '_modified_data', '_old', '_query', '_serialize_execution_settings', '_serialize_inputs', '_set', '_to_api_file_format', 'abort', 'app', 'batch', 'batch_by', 'batch_group', 'batch_input', 'bulk_get', 'clone', 'create', 'created_by', 'created_time', 'deepcopy', 'delete', 'description', 'end_time', 'equals', 'errors', 'executed_by', 'execution_settings', 'execution_status', 'field', 'get', 'get_batch_children', 'get_execution_details', 'href', 'id', 'inputs', 'name', 'outputs', 'parent', 'pric

(Pdb) p task_info.get_execution_details().jobs['sambamba_merge']
*** TypeError: list indices must be integers or slices, not str
(Pdb)  p task_info.get_execution_details().jobs[-1]
<Job: name=sambamba_merge, status=FAILED>
(Pdb) p dir(task_info.get_execution_details().jobs[-1])
['_API', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_data', '_dirty', '_fields', '_modified_data', '_old', '_query', '_set', 'command_line', 'deepcopy', 'delete', 'docker', 'end_time', 'equals', 'field', 'get', 'instance', 'logs', 'name', 'reload', 'retried', 'start_time', 'status']
(Pdb) p task_info.get_execution_details().jobs[-1].name
'sambamba_merge'
(Pdb) p task_info.get_execution_details().jobs[-1].log

BdbQuit: 